# Doc2VecでWikipedia文書を使用し、マイオペのデータで予測

巨大コーパスをもつWiki文書の一部を使用して、Wiki文書でボキャブラリを作成し、マイオペの学習セットで学習／予測させます。

PV-DBOW手法であれば、マイオペの学習セットを混ぜて学習させることで、マイオペのデータでも予測ができることが確認できました。

## (1) Wikipediaコンテンツファイルから全文書を抽出

<a href="31-Wikipedia-contents-csv.ipynb"><b>こちらの手順</b></a> にて、いったんローカルPCにCSVファイル化しておきます。



## (2) TaggedDocumentを生成

### (2-1) Wikipedia文書のTaggedDocumentを生成

Wikipedia文書のCSVファイルからデータフレームを読み込み、TaggedDocumentに変換します。

In [1]:
'''
    環境準備
'''
import sys
import os

import numpy as np
import pandas as pd
 
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [3]:
from gensim.models.doc2vec import TaggedDocument
# from learning.core.nlang import Nlang

def get_tagged_document_list(wiki_ids, wiki_contents):
    '''
        タグとコーパスをTaggedDocumentに設定
    '''
    tagged_document_list = []

    for index, _ in enumerate(wiki_ids):
        tag = wiki_ids[index]
        sentences = wiki_contents[index]

        # 品詞は落とさない様にする
        separated_sentences = Nlang_naive.split(sentences)
        words = separated_sentences.split()
        tagged_document_list.append(TaggedDocument(words=words, tags=[tag]))
        
    return tagged_document_list

In [4]:
'''
    Wikipedia文書から生成されたCSVを読み込み、
    配列を生成する
'''
extracted_dir_path = '/Users/makmorit/Documents/Development/Wikipedia/extracted'
csv_file_name = os.path.join(extracted_dir_path, 'Wikipedia-content.csv')
df = pd.read_csv(csv_file_name)

_wiki_ids = np.array(df['id'])
_wiki_titles = np.array(df['title'])
_wiki_contents = np.array(df['content'])

In [5]:
'''
    時間がかかるので一部のみ使用します（全量＝100万件）
'''
partial_cnt = 1000

wiki_ids = _wiki_ids[0:partial_cnt]
wiki_contents = _wiki_contents[0:partial_cnt]

In [6]:
wiki_tagged_document_list = get_tagged_document_list(wiki_ids, wiki_contents)

In [7]:
len(wiki_tagged_document_list)

1000

### (2-2) マイオペ文書のTaggedDocumentを生成

In [9]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 11 # benefitone.csv
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}
learning_parameter = LearningParameter(attr)

In [10]:
'''
    マイオペの学習セットを取得
'''
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])
ids = np.array(learning_training_messages['id'])

2017/05/26 AM 10:12:51 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/26 AM 10:12:51 ['./fixtures/question_answers/toyotsu_human.csv']


In [22]:
'''
    既存セットの否定の意味を持つトレーニングセットを追加します。
    例）
    　「契約書を見たいのですが（正解＝4683）」
    　に対して、否定の意味を持つセットは下記の通り。
    　'契約書を確認したくないのですが'
    　'契約書を確認したくない'
    　'契約書を閲覧したくない'
    　'契約書が見たくないのですが、どこにありますか？'
    　'契約書が見たくない'
'''
questions_nega = np.array([
    '契約書を確認したくないのですが',
    '契約書を確認したくない',
    '契約書を閲覧したくない',
    '契約書が見たくないのですが、どこにありますか？',
    '契約書が見たくない'
])
questions = np.append(questions, questions_nega)

In [24]:
answer_ids_nega = np.array([
    5500,
    5500,
    5500,
    5500,
    5500
])
answer_ids = np.append(answer_ids, answer_ids_nega)

In [25]:
ids_nega = np.array([
    1903000,
    1903001,
    1903002,
    1903003,
    1903004
])
ids = np.append(ids, ids_nega)

In [26]:
'''
    正しく要素が追加されたかどうかチェックします。
'''
print('count: questions=%d, answer_ids=%d (class count=%d), ids=%d' % (
    np.size(questions), np.size(answer_ids), np.size(np.unique(answer_ids)), np.size(ids)
))

count: questions=7088, answer_ids=7088 (class count=89), ids=7088


In [27]:
'''
    マイオペの学習セットから、コーパスを生成
    
    タグとして、learning_training_messages.id を付与
'''
myope_tagged_document_list = get_tagged_document_list(ids, questions)
len(myope_tagged_document_list)

7088

### (2-2) Wikipedia文書とマイオペ文書の、両方のTaggedDocumentを合体させる

In [28]:
all_tagged_document_list = []
all_tagged_document_list.extend(wiki_tagged_document_list)
all_tagged_document_list.extend(myope_tagged_document_list)

In [29]:
len(all_tagged_document_list)

8088

## (3) 学習実行

### (3-1) TaggedDocumentをインプットとして学習

In [30]:
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wikipedia.PV%d.model' % dm
    return model_path

def build_vocabulary(tagged_document_list, dm=0, size=100):
    '''
        tagged_document_listは、
        Wikipedia文書のタグ付きコーパス
        (前述のtagged_document)が収容されたリスト。

        これを引数にして、
        ボキャブラリ生成を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
    '''
    model = Doc2Vec(dm=dm, size=size, min_count=1, iter=size*10)
    model.build_vocab(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('build_vocabulary: model saved temporary')

    return model

def train(model, tagged_document_list, dm=0):
    '''
        tagged_documentが収容されたリストを引数にして、
        学習を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
        （今回は比較のため、両方で試す）
    '''
    ret = model.train(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('train: document vector size=%d, return=%d' % (len(model.docvecs), ret))

    return ret

In [31]:
'''
    tagged_document_listを使用し、学習実行（PV-DBOW）
'''
model_dbow = build_vocabulary(all_tagged_document_list, dm=0, size=200)

build_vocabulary: model saved temporary


In [32]:
train(model_dbow, all_tagged_document_list, dm=0) 

train: document vector size=8088, return=2472743229


2472743229

In [42]:
'''
    tagged_document_listを使用し、学習実行（PV-DM）
'''
model_dm = build_vocabulary(all_tagged_document_list, dm=1, size=200)

build_vocabulary: model saved temporary


In [43]:
train(model_dm, all_tagged_document_list, dm=1) 

train: document vector size=8088, return=2472729001


2472729001

## (3) 予測実行

作成されたモデルを使用して予測を実行する関数になります。

In [33]:
from gensim import models

def predict(word, dm=0):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(doc2vec_model_path(dm))
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

def get_title_from_tag(tag):
    '''
        tag に対応するWikipediaのタイトルを取得。

        ただし、合致したものがなければ、
        マイオペの id と判断し、
        id に対応する answer_id を取得する。
    '''
    for index, _ in enumerate(wiki_ids):
        if wiki_ids[index] == tag:
            return _wiki_titles[index]

    for index, _ in enumerate(ids):
        if ids[index] == tag:
            return answer_ids[index]
        
    return None

def print_predict_result(corpus, similarities):
    print(corpus)
    print("id, answer_id, similarity")

    for tag, similarity in similarities:
        print(tag, get_title_from_tag(tag), similarity)

### (3-1) 適当な質問文で予測（PV-DBOW)

#### 正解があるものは高めの類似度を示します

In [34]:
'''
    契約書を見たいのですが（正解＝4683）
'''
corpus, ret = predict('契約書を見たいのですが', dm=0)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見る', 'たい', 'の', 'です', 'が']
id, answer_id, similarity
1898658 4683 0.6546140909194946
1895471 4683 0.6474508047103882
1901816 4683 0.636380672454834
1895051 4683 0.5945093631744385
1898648 4683 0.5925205945968628
1895461 4683 0.5893855094909668
1903004 5500 0.5828196406364441
1898656 4683 0.5741859674453735
1895466 4683 0.5703336000442505
1895469 4683 0.5700043439865112


In [35]:
'''
    EXカードを貸してください（正解＝4678）
'''
corpus, ret = predict('EXカードを貸してください', dm=0)
print_predict_result(corpus, ret)

['ＥＸ', 'カード', 'を', '貸す', 'て', 'くださる']
id, answer_id, similarity
1899813 4678 0.7427541613578796
1896626 4678 0.7213946580886841
1898215 4678 0.7042893171310425
1901396 4678 0.7013541460037231
1901402 4678 0.7011991143226624
1898209 4678 0.693244457244873
1899807 4678 0.676848292350769
1896087 4742 0.654140830039978
1899274 4742 0.6531285643577576
1901397 4678 0.6529818177223206


In [36]:
'''
    正解に、肯定＋否定のデータセットがあるもの ---> 否定の品詞を落とさない効果が出た様子。
    
    （「契約書を見たいのですが」の正解＝4683、否定データセットの正解＝5500）
'''
corpus, ret = predict('契約書を見せたくないのですが', dm=0)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見せる', 'たい', 'ない', 'の', 'です', 'が']
id, answer_id, similarity
1903000 5500 0.5620639324188232
1895470 4683 0.5539571642875671
1898648 4683 0.5516992807388306
1898658 4683 0.5513678789138794
1898656 4683 0.5484150648117065
1895466 4683 0.5476746559143066
1898654 4683 0.5464545488357544
1895469 4683 0.5444513559341431
1895461 4683 0.5439090728759766
1898653 4683 0.5425601601600647


#### 正解があるものについて、言い回しを変えてみます。

想定される結果からは、かけ離れたものになってしまいます。

ちなみに id="1902121" の文書は「メーリングリストを作成したい」

（「作成」と「したい」にかかっているものと考えられます）

In [37]:
'''
    正解の別な言い回し（教師データにない）---> 別の答えが返ってきてしまう

    （「契約書を見たいのですが」の正解＝4683）
'''
corpus, ret = predict('契約を結んだ時に作成される文書を確認したい', dm=0)
print_predict_result(corpus, ret)

['契約', 'を', '結ぶ', 'だ', '時', 'に', '作成', 'する', 'れる', '文書', 'を', '確認', 'する', 'たい']
id, answer_id, similarity
1902121 4797 0.5602943897247314
1895356 4797 0.5598479509353638
1901225 4678 0.5534567832946777
1895466 4683 0.5527678728103638
1901231 4678 0.5504263639450073
1898656 4683 0.5500228404998779
1895469 4683 0.5491943359375
1898653 4683 0.5456653833389282
1900660 4678 0.5446206331253052
1900376 4678 0.5444127321243286


#### 正解がないものは低めの類似度を示します

In [38]:
'''
    おいしいラーメンが食べたいです（正解＝なし）
'''
corpus, ret = predict('おいしいラーメンが食べたいです', dm=0)
print_predict_result(corpus, ret)

['おいしい', 'ラーメン', 'が', '食べる', 'たい', 'です']
id, answer_id, similarity
1900006 4678 0.5084030628204346
1896819 4678 0.4991321563720703
1900001 4678 0.49748605489730835
1896814 4678 0.49560075998306274
1897410 4678 0.4940197467803955
1900597 4678 0.4930015504360199
1900290 4783 0.49278855323791504
1897103 4783 0.48947274684906006
1897112 4783 0.4881078600883484
1900018 4678 0.48769503831863403


In [39]:
'''
    無関係な質問をしてみます。
'''
corpus, ret = predict('風邪を引いた時の治療薬を教えてください', dm=0)
print_predict_result(corpus, ret)

['風邪', 'を', '引く', 'た', '時', 'の', '治療', '薬', 'を', '教える', 'て', 'くださる']
id, answer_id, similarity
1895436 4677 0.5153379440307617
1898670 4686 0.5125332474708557
1898623 4677 0.510613203048706
1895483 4686 0.5096703171730042
1895470 4683 0.4902334213256836
1898389 4798 0.48658186197280884
1898657 4683 0.48633667826652527
1898617 4677 0.4823513627052307
1895430 4677 0.4786865711212158
1896030 4678 0.47626757621765137


#### マイオペとは無関係な、Wikipedia文書に関するデータの予測

なぜか、マイオペの文書が帰ってきてしまいます。ただし類似度は低めです。

類似度が一番高いのはマイオペの id=1900198（洗剤をください。）という質問文でした。

In [40]:
'''
    Wikiタイトルに近い文言で、Google検索っぽく質問をしてみます。
'''
corpus, ret = predict('風邪', dm=0)
print_predict_result(corpus, ret)

['風邪']
id, answer_id, similarity
1900198 4678 0.5264950394630432
1897011 4678 0.5087937116622925
1899852 4678 0.5072660446166992
1899858 4678 0.5061943531036377
1900306 4678 0.50048828125
1899864 4678 0.4985775947570801
1900232 4678 0.4971644878387451
1897008 4678 0.49680688977241516
1900195 4678 0.4957461357116699
1897045 4678 0.493877112865448


#### Wiki 文書の一部から抜粋した文章で予測

なぜか、マイオペの文書が帰ってきてしまいます。ただし類似度は低めです。

類似度が一番高いのはマイオペの id=1899983（紐が見当たりません。）という質問文でした。

In [41]:
'''
    Wiki本文に近い文言で、質問をしてみます。
'''
wiki_like_sentence = '''漁業にはさまざまな人々が関わっているが、漁撈活動に専業として携わる者のことは漁師という。
'''
corpus, ret = predict(wiki_like_sentence, dm=0)
print_predict_result(corpus, ret)

['漁業', 'に', 'は', 'さまざま', 'だ', '人々', 'が', '関わる', 'て', 'いる', 'が', '漁撈', '活動', 'に', '専業', 'として', '携わる', '者', 'の', 'こと', 'は', '漁師', 'と', 'いう']
id, answer_id, similarity
1899983 4678 0.4190804362297058
1898176 4678 0.4167530834674835
1901363 4678 0.41559410095214844
1900217 4678 0.41466325521469116
1899433 4751 0.4121313691139221
1899869 4678 0.4113248586654663
1902069 4678 0.41029930114746094
1899863 4678 0.40991640090942383
1899982 4678 0.40940573811531067
1901360 4678 0.4093107581138611


### (3-2) 適当な質問文で予測（PV-DM)

#### 正解があるもの

最初の質問に対しては誤答されてしまいます（id="1901615" は「動画を見たい」）

In [44]:
'''
    契約書を見たいのですが（正解＝4683）
'''
corpus, ret = predict('契約書を見たいのですが', dm=1)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見る', 'たい', 'の', 'です', 'が']
id, answer_id, similarity
1901615 4808 0.6618736982345581
1898428 4808 0.6436389088630676
1901931 4781 0.6276744604110718
1901816 4683 0.6150595545768738
1899889 4678 0.6101311445236206
1900435 4678 0.6081597805023193
1901870 4678 0.6015615463256836
1900367 4678 0.6012057065963745
1898658 4683 0.599880576133728
1899786 4781 0.5995423793792725


こちらはかろうじて正解でした。ただし類似度は低いです。

In [45]:
'''
    EXカードを貸してください（正解＝4678）
'''
corpus, ret = predict('EXカードを貸してください', dm=1)
print_predict_result(corpus, ret)

['ＥＸ', 'カード', 'を', '貸す', 'て', 'くださる']
id, answer_id, similarity
1901053 4678 0.5373465418815613
1901148 4678 0.5133180022239685
1899051 4733 0.5031865239143372
1900493 4678 0.5002593994140625
1899667 4678 0.49830618500709534
1899664 4678 0.49728626012802124
1899039 4678 0.49396175146102905
1902072 4678 0.4921454191207886
1902095 4804 0.4878588914871216
1899072 4678 0.48723965883255005


こちらは正解できませんでした。（id="1898709" は「小口申請をしたいのですが」）

In [46]:
'''
    正解に、肯定＋否定のデータセットがあるもの
    
    （「契約書を見たいのですが」の正解＝4683、否定データセットの正解＝5500）
'''
corpus, ret = predict('契約書を見せたくないのですが', dm=1)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見せる', 'たい', 'ない', 'の', 'です', 'が']
id, answer_id, similarity
1898709 4692 0.5587493181228638
1898448 4808 0.5576707720756531
1901635 4808 0.5570188760757446
1901831 4710 0.5560088157653809
1899540 4678 0.5535436868667603
1901281 4678 0.5519620180130005
1899560 4678 0.5486915707588196
1899603 4678 0.5460902452468872
1896373 4678 0.532397449016571
1895522 4692 0.531071662902832


#### 正解があるものについて、言い回しを変えてみます。

想定される結果からは、かけ離れたものになってしまいます。（id="1898948" は「会議室を使いたい」）

In [47]:
'''
    正解の別な言い回し（教師データにない）---> 別の答えが返ってきてしまう

    （「契約書を見たいのですが」の正解＝4683）
'''
corpus, ret = predict('契約を結んだ時に作成される文書を確認したい', dm=1)
print_predict_result(corpus, ret)

['契約', 'を', '結ぶ', 'だ', '時', 'に', '作成', 'する', 'れる', '文書', 'を', '確認', 'する', 'たい']
id, answer_id, similarity
1898948 4729 0.5893667936325073
1900102 4678 0.5748825073242188
1901851 4678 0.5745905041694641
1898961 4730 0.5703769326210022
1900372 4678 0.5684346556663513
1901870 4678 0.5682883262634277
1898946 4729 0.5670807361602783
1898945 4729 0.5651745796203613
1900269 4678 0.5625522136688232
1895824 4678 0.5619271397590637


#### 正解がないもの

類似文書がない場合は、いくつかWikiの文書から類似しているものを引っ張り出してくる様です。

（Wikiからは、doc id="901" の「旭川市」というタイトルがついた文書を引っ張ってきています）

ただし類似度はかなり低いです。

In [48]:
'''
    おいしいラーメンが食べたいです（正解＝なし）
'''
corpus, ret = predict('おいしいラーメンが食べたいです', dm=1)
print_predict_result(corpus, ret)

['おいしい', 'ラーメン', 'が', '食べる', 'たい', 'です']
id, answer_id, similarity
901 旭川市 0.0029766522347927094
649 すがやみつる -0.0033093495294451714
1151 荒俣宏 -0.028376499190926552
952 流水凜子 -0.02860189415514469
609 大島やすいち -0.04567909240722656
1225 L4 -0.05281950533390045
758 小谷憲一 -0.05302031338214874
1375 槇村さとる -0.05659448355436325
920 田島みるく -0.05930089205503464
623 奥田ひとし -0.060799650847911835


類似マイオペ文書がある場合は適当な文書が戻ります。（id="1901441" は「PCが壊れた」）

確かに、何かが壊れたことには変わりありませんが・・・

In [49]:
'''
    無関係な質問をしてみます。
'''
corpus, ret = predict('風邪を引いた時の治療薬を教えてください', dm=1)
print_predict_result(corpus, ret)

['風邪', 'を', '引く', 'た', '時', 'の', '治療', '薬', 'を', '教える', 'て', 'くださる']
id, answer_id, similarity
1901441 4802 0.3792627453804016
1901686 4808 0.3756944239139557
1897212 4678 0.3739567995071411
1898943 4729 0.3660581111907959
1900399 4678 0.36351415514945984
1898704 4692 0.3627987504005432
1900082 4678 0.3625839948654175
1900396 4678 0.36191168427467346
1899690 4678 0.36137229204177856
1901346 4678 0.3611118793487549


#### マイオペとは無関係な、Wikipedia文書に関するデータの予測

Wikiの文書から、doc id="342" の「桜野みねね」というタイトルがついた文書を引っ張ってきています。これは風邪とは何の関係もない文書です。

ただし類似度が一番高いのはマイオペの id=1898798（複合機が壊れた）という質問文でした。

In [50]:
'''
    Wikiタイトルに近い文言で、Google検索っぽく質問をしてみます。
'''
corpus, ret = predict('風邪', dm=1)
print_predict_result(corpus, ret)

['風邪']
id, answer_id, similarity
1898798 4709 0.4264153838157654
1900055 4678 0.426098495721817
1901898 4755 0.4252046048641205
342 桜野みねね 0.4239950180053711
1899467 4755 0.42048949003219604
1900065 4678 0.4203770160675049
1901901 4758 0.4192509055137634
1900910 4678 0.41891440749168396
1902076 4678 0.4172428846359253
1901821 4692 0.413732647895813


#### Wiki 文書の一部から抜粋した文章で予測

なぜかマイオペの文書が帰ってしまいます。

類似度が一番高いのはマイオペの id=1901490（表計算ソフトについて）という質問文でした。

In [51]:
'''
    Wiki本文に近い文言で、質問をしてみます。
'''
wiki_like_sentence = '''漁業にはさまざまな人々が関わっているが、漁撈活動に専業として携わる者のことは漁師という。
'''
corpus, ret = predict(wiki_like_sentence, dm=1)
print_predict_result(corpus, ret)

['漁業', 'に', 'は', 'さまざま', 'だ', '人々', 'が', '関わる', 'て', 'いる', 'が', '漁撈', '活動', 'に', '専業', 'として', '携わる', '者', 'の', 'こと', 'は', '漁師', 'と', 'いう']
id, answer_id, similarity
1901490 4821 0.48075902462005615
1901496 4821 0.4784567356109619
1898303 4821 0.47229474782943726
1901481 4821 0.4705294072628021
1898309 4821 0.4651545286178589
1902100 4821 0.4561212658882141
1899190 4678 0.44923144578933716
1901495 4821 0.4475501775741577
1901489 4821 0.4471065402030945
1899845 4678 0.44237256050109863
